In [70]:
import pandas as pd
import numpy as np

from utils import process_data
from utils import read_batched_data

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import make_scorer, roc_auc_score

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import pickle

import matplotlib.pyplot as plt

from collections import defaultdict

In [2]:
data_dict = process_data('normal', folds = 5)

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

Beginning data processing ...
Reading in batched data ...


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:16<00:00,  1.15it/s]


Splitting data into train and test ...
Scaling data ...
Completed normal data processing.


## Neural Network Model

In [6]:
def build_NN(num_classes, hidden_layers, neurons_per_layer, dropout_rate, input_dimensions):
    model = Sequential()
    model.add(Dense(neurons_per_layer, input_dim = input_dimensions, activation = 'relu'))

    for i in range(hidden_layers):
        model.add(Dense(neurons_per_layer, activation = 'relu'))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    model.add(Dense(num_classes, activation = 'softmax'))

    model.compile(
        loss = 'sparse_categorical_crossentropy',
        optimizer = 'adam',
        metrics = [tensorflow.keras.metrics.SparseCategoricalAccuracy()]
    )
    
    return model

In [12]:
roc = make_scorer(roc_auc_score, needs_proba = True)
folds = 5

dropout_rates = [0, 0.2]
batch_size = [32]
epochs = [10]
hyperparameters = [
    {
        'hidden_layers': [1],
        'neurons_per_layer': [4, 8, 16, 32, 64],
        'dropout_rate': dropout_rates,
        'batch_size': batch_size,
        'epochs': epochs,
    },
    {
        'hidden_layers': [2],
        'neurons_per_layer': [4, 8, 16, 32],
        'dropout_rate': dropout_rates,
        'batch_size': batch_size,
        'epochs': epochs,
    },
    {
        'hidden_layers': [3],
        'neurons_per_layer': [4, 8, 16],
        'dropout_rate': dropout_rates,
        'batch_size': batch_size,
        'epochs': epochs,
    },
#     {
#         'hidden_layers': [4],
#         'neurons_per_layer': [4, 8],
#         'dropout_rate': dropout_rates,
#         'batch_size': batch_size,
#         'epochs': epochs,
#     },
#     {
#         'hidden_layers': [5],
#         'neurons_per_layer': [4],
#         'dropout_rate': dropout_rates,
#         'batch_size': batch_size,
#         'epochs': epochs,
#     }
]

num_classes = max(data_dict['y_train'])+1
input_dimensions = data_dict['X_train_scaled'].shape[1]

nn_model = KerasClassifier(build_fn = build_NN, num_classes = num_classes, input_dimensions = input_dimensions)
kfolds = StratifiedKFold(n_splits = folds)

grid = GridSearchCV(nn_model, param_grid = hyperparameters, cv = kfolds, scoring = roc, verbose = 10, n_jobs = 1)
model = grid.fit(data_dict['X_train_scaled'], data_dict['y_train'])


<ipython-input-12-4ab59ca7dc5b>:48: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  nn_model = KerasClassifier(build_fn = build_NN, num_classes = num_classes, input_dimensions = input_dimensions)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5; 1/24] START batch_size=32, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=4
Epoch 1/10
72349/72349 [==============================] - 39s 529us/step - loss: 0.0042 - sparse_categorical_accuracy: 0.9992
Epoch 2/10
72349/72349 [==============================] - 39s 537us/step - loss: 0.0028 - sparse_categorical_accuracy: 0.9992
Epoch 3/10
72349/72349 [==============================] - 37s 511us/step - loss: 0.0028 - sparse_categorical_accuracy: 0.9992
Epoch 4/10
72349/72349 [==============================] - 37s 512us/step - loss: 0.0028 - sparse_categorical_accuracy: 0.9992
Epoch 5/10
72349/72349 [==============================] - 38s 532us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9992
Epoch 6/10
72349/72349 [==============================] - 37s 515us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9992
Epoch 7/10
72349/72349 [==============================] - 37s 515us/step - loss: 0

72349/72349 [==============================] - 41s 568us/step - loss: 0.0026 - sparse_categorical_accuracy: 0.9995
[CV 1/5; 3/24] END batch_size=32, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=16;, score=0.989 total time= 7.0min
[CV 2/5; 3/24] START batch_size=32, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=16
Epoch 1/10
72349/72349 [==============================] - 41s 557us/step - loss: 0.0032 - sparse_categorical_accuracy: 0.9993
Epoch 2/10
72349/72349 [==============================] - 40s 553us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
72349/72349 [==============================] - 40s 549us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 4/10
72349/72349 [==============================] - 40s 552us/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9995
Epoch 5/10
72349/72349 [==============================] - 40s 551us/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9995
Epoch 6/10
72349/72349

72349/72349 [==============================] - 41s 573us/step - loss: 0.0028 - sparse_categorical_accuracy: 0.9995
Epoch 9/10
72349/72349 [==============================] - 41s 573us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9995
Epoch 10/10
72349/72349 [==============================] - 42s 575us/step - loss: 0.0031 - sparse_categorical_accuracy: 0.9995
[CV 2/5; 5/24] END batch_size=32, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=64;, score=0.983 total time= 7.0min
[CV 3/5; 5/24] START batch_size=32, dropout_rate=0, epochs=10, hidden_layers=1, neurons_per_layer=64
Epoch 1/10
72349/72349 [==============================] - 41s 560us/step - loss: 0.0029 - sparse_categorical_accuracy: 0.9995
Epoch 2/10
72349/72349 [==============================] - 41s 561us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
72349/72349 [==============================] - 41s 560us/step - loss: 0.0026 - sparse_categorical_accuracy: 0.9995
Epoch 4/10
72349/7234

72349/72349 [==============================] - 40s 549us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 7/10
72349/72349 [==============================] - 40s 550us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 8/10
72349/72349 [==============================] - 40s 558us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 9/10
72349/72349 [==============================] - 40s 551us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 10/10
72349/72349 [==============================] - 40s 550us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
[CV 3/5; 7/24] END batch_size=32, dropout_rate=0.2, epochs=10, hidden_layers=1, neurons_per_layer=8;, score=0.985 total time= 6.8min
[CV 4/5; 7/24] START batch_size=32, dropout_rate=0.2, epochs=10, hidden_layers=1, neurons_per_layer=8
Epoch 1/10
72349/72349 [==============================] - 41s 555us/step - loss: 0.0058 - sparse_categorical_accuracy: 0.9984
Epoch 2/10
72349/72

...

Epoch 1/10
72349/72349 [==============================] - 46s 630us/step - loss: 0.0033 - sparse_categorical_accuracy: 0.9994
Epoch 2/10
72349/72349 [==============================] - 45s 623us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
72349/72349 [==============================] - 45s 626us/step - loss: 0.0026 - sparse_categorical_accuracy: 0.99950s - loss: 0.0026 - sparse_categorical_accuracy: 0.999
Epoch 4/10
72349/72349 [==============================] - 45s 623us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9995
Epoch 5/10
72349/72349 [==============================] - 45s 623us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9995
Epoch 6/10
72349/72349 [==============================] - 45s 625us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9995
Epoch 7/10
72349/72349 [==============================] - 45s 626us/step - loss: 0.0027 - sparse_categorical_accuracy: 0.9995
Epoch 8/10
72349/72349 [==============================] - 45s 62

72349/72349 [==============================] - 47s 656us/step - loss: 0.0031 - sparse_categorical_accuracy: 0.9995
Epoch 7/10
72349/72349 [==============================] - 48s 659us/step - loss: 0.0033 - sparse_categorical_accuracy: 0.9995
Epoch 8/10
72349/72349 [==============================] - 48s 657us/step - loss: 0.0029 - sparse_categorical_accuracy: 0.9995
Epoch 9/10
72349/72349 [==============================] - 48s 658us/step - loss: 0.0033 - sparse_categorical_accuracy: 0.9995
Epoch 10/10
72349/72349 [==============================] - 48s 657us/step - loss: 0.0031 - sparse_categorical_accuracy: 0.9995
[CV 3/5; 18/24] END batch_size=32, dropout_rate=0.2, epochs=10, hidden_layers=2, neurons_per_layer=32;, score=0.986 total time= 8.1min
[CV 4/5; 18/24] START batch_size=32, dropout_rate=0.2, epochs=10, hidden_layers=2, neurons_per_layer=32
Epoch 1/10
72349/72349 [==============================] - 48s 651us/step - loss: 0.0034 - sparse_categorical_accuracy: 0.9994
Epoch 2/10
7234

Epoch 1/10
72349/72349 [==============================] - 40s 545us/step - loss: 0.0044 - sparse_categorical_accuracy: 0.9991
Epoch 2/10
72349/72349 [==============================] - 40s 547us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
72349/72349 [==============================] - 39s 544us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 4/10
72349/72349 [==============================] - 39s 543us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 5/10
72349/72349 [==============================] - 39s 542us/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9995
Epoch 6/10
72349/72349 [==============================] - 39s 544us/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9995
Epoch 7/10
72349/72349 [==============================] - 39s 543us/step - loss: 0.0022 - sparse_categorical_accuracy: 0.9995
Epoch 8/10
72349/72349 [==============================] - 39s 543us/step - loss: 0.0022 - sparse_categorical_accuracy:

Epoch 1/10
72349/72349 [==============================] - 44s 595us/step - loss: 0.0031 - sparse_categorical_accuracy: 0.9994
Epoch 2/10
72349/72349 [==============================] - 43s 592us/step - loss: 0.0025 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
72349/72349 [==============================] - 43s 598us/step - loss: 0.0026 - sparse_categorical_accuracy: 0.9995
Epoch 4/10
72349/72349 [==============================] - 43s 596us/step - loss: 0.0024 - sparse_categorical_accuracy: 0.9995
Epoch 5/10
72349/72349 [==============================] - 43s 593us/step - loss: 0.0025 - sparse_categorical_accuracy: 0.9995
Epoch 6/10
72349/72349 [==============================] - 43s 595us/step - loss: 0.0031 - sparse_categorical_accuracy: 0.9995
Epoch 7/10
72349/72349 [==============================] - 43s 594us/step - loss: 0.0028 - sparse_categorical_accuracy: 0.9995
Epoch 8/10
72349/72349 [==============================] - 43s 594us/step - loss: 0.0025 - sparse_categorical_accuracy:

Epoch 1/10
72349/72349 [==============================] - 48s 660us/step - loss: 0.0041 - sparse_categorical_accuracy: 0.9992
Epoch 2/10
72349/72349 [==============================] - 48s 661us/step - loss: 0.0030 - sparse_categorical_accuracy: 0.9995
Epoch 3/10
72349/72349 [==============================] - 48s 666us/step - loss: 0.0030 - sparse_categorical_accuracy: 0.9995
Epoch 4/10
72349/72349 [==============================] - 48s 660us/step - loss: 0.0031 - sparse_categorical_accuracy: 0.9995
Epoch 5/10
72349/72349 [==============================] - 48s 662us/step - loss: 0.0034 - sparse_categorical_accuracy: 0.9994
Epoch 6/10
72349/72349 [==============================] - 48s 662us/step - loss: 0.0033 - sparse_categorical_accuracy: 0.9994
Epoch 7/10
72349/72349 [==============================] - 48s 670us/step - loss: 0.0036 - sparse_categorical_accuracy: 0.9994
Epoch 8/10
72349/72349 [==============================] - 48s 661us/step - loss: 0.0036 - sparse_categorical_accuracy:

In [27]:
nn_results = {
    'cv_results_': grid.cv_results_,
    'best_score_': grid.best_score_,
    'best_params_': grid.best_params_,
    'best_index_': grid.best_index_,
    'scorer_': grid.scorer_,
    'n_splits_': grid.n_splits_,
}

In [77]:
cv_results = defaultdict(list)
for i, mean_test_score in enumerate(grid.cv_results_['mean_test_score']):
    cv_results['dropout_rate'].append(grid.cv_results_['params'][i]['dropout_rate'])
    cv_results['hidden_layers'].append(grid.cv_results_['params'][i]['hidden_layers'])
    cv_results['neurons_per_layer'].append(grid.cv_results_['params'][i]['neurons_per_layer'])
    cv_results['mean_test_score'].append(mean_test_score)
    
nn_cv_results_df = pd.DataFrame(cv_results)
display(nn_cv_results_df)

,dropout_rate,hidden_layers,neurons_per_layer,mean_test_score
0,0.0,1,4,0.990466
1,0.0,1,8,0.990546
2,0.0,1,16,0.987941
3,0.0,1,32,0.987033
4,0.0,1,64,0.985278
5,0.2,1,4,0.988553
6,0.2,1,8,0.989101
7,0.2,1,16,0.989117
8,0.2,1,32,0.984063
9,0.2,1,64,0.985035


In [29]:
grid.best_score_

0.9907930249520142

In [28]:
with open('neural-network-gridsearchcv-results.pkl', 'wb') as handle:
    pickle.dump(nn_results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# final_model = build_NN(num_classes, model.best_params_['hidden_layers'], model.best_params_['neurons_per_layer'], model.best_params_['dropout_rate'], input_dimensions)
# final_model.fit(x_train, y_train, batch_size = model.best_params_['batch_size'], epochs = model.best_params_['epochs'])

# with open('models/model_nn.sav', 'wb') as handle:
#     pickle.dump(final_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
final_model = build_NN(num_classes, model.best_params_['hidden_layers'], model.best_params_['neurons_per_layer'], model.best_params_['dropout_rate'], input_dimensions)
final_model.fit(x_train, y_train, batch_size = model.best_params_['batch_size'], epochs = model.best_params_['epochs'])

In [40]:
with open('neural-network-model.json', 'w') as json_file:
    json_file.write(grid.best_estimator_.model.to_json())
grid.best_estimator_.model.save_weights('neural-network-model.h5')